__LIWC__

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import re
from time import time

In [3]:
from sklearn.svm import LinearSVC, SVC
from sklearn import decomposition
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.utils.multiclass import unique_labels
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [4]:
def split_train_test_data(input_data, target, ratio=0.3, rand_state=42):
    return train_test_split(input_data, target, test_size=ratio, stratify=target, random_state=rand_state)

def apply_grid_search_cv(pipe, param_grid, X_train, y_train, X_test, y_test, print_flag=True, score_matrix=f1_score, n_jobs=-1, cv=5):
    grid_search = GridSearchCV(pipe, param_grid=param_grid, scoring=make_scorer(score_matrix), n_jobs=n_jobs, cv=cv)
    t0 = time()
    res = grid_search.fit(X_train, y_train)
    if print_flag:
        print("done in %0.3fs" % (time() - t0))
        print("best params:")
        print(res.best_params_)
        print("Best cross-validation score: {:.3f}".format(grid_search.best_score_))
        print("Test-set score: {:.3f}".format(grid_search.score(X_test, y_test)))        
    return grid_search

def save_class_report_cv(grid_search, X_test, y_test, target_names, filename):
    y_pred = grid_search.best_estimator_.predict(X_test)
    report = classification_report(y_test, y_pred, target_names=target_names, output_dict=True)
    df = pd.DataFrame(report).transpose()
    df.to_csv(filename)
    return df

In [5]:
def save_cross_validation_results(grid_search, filename, print_flag=True):
    param_keys = list(grid_search.cv_results_["params"][0].keys())
    matrix_list = ["mean_test_score", "std_test_score", "rank_test_score", "mean_fit_time"] 
    col = param_keys + matrix_list

    cv_results = []
    cv_results.append(col)    
    
    for param, score, std, rank, time in zip(grid_search.cv_results_["params"], grid_search.cv_results_["mean_test_score"],grid_search.cv_results_["std_test_score"],
                                             grid_search.cv_results_["rank_test_score"], grid_search.cv_results_["mean_fit_time"]):
        row_item = list(param.values())
        row_item.append(score)
        row_item.append(std)
        row_item.append(rank)
        row_item.append(time)
        cv_results.append(row_item)
        
    cv_results = pd.DataFrame(cv_results) 
    header = cv_results.iloc[0] 
    cv_results = cv_results[1:]
    cv_results = cv_results.rename(columns = header)
    cv_results = cv_results.sort_values(by=['rank_test_score'])
    cv_results.to_csv(filename)
    if print_flag:
        print(cv_results.head(6))
#        print(cv_results.tail(6))

In [6]:
hotels = pd .read_csv("data/LIWC2015_mod5.csv")

In [7]:
pd.set_option('display.max_columns', 500)

In [8]:
hotels = hotels.drop(hotels.columns[0], axis=1)

In [9]:
hotels.describe(include="all")

,deceptive,hotel,polarity,source,text,text_length,lower_case,lc_no_punct,norm,norm_lemma,norm_stem,norm_lemma_stopword,norm_stem_stopword,ADJ_count,ADP_count,ADV_count,AUX_count,CCONJ_count,DET_count,INTJ_count,NOUN_count,NUM_count,PART_count,PRON_count,PROPN_count,PUNCT_count,SCONJ_count,SYM_count,VERB_count,X_count,class,WC,Analytic,Clout,Authentic,Tone,WPS,Sixltr,Dic,function,pronoun,ppron,i,we,you,shehe,they,ipron,article,prep,auxverb,adverb,conj,negate,verb,adj,compare,interrog,number,quant,affect,posemo,negemo,anx,anger,sad,social,family,friend,female,male,cogproc,insight,cause,discrep,tentat,certain,differ,percept,see,hear,feel,bio,body,health,sexual,ingest,drives,affiliation,achieve,power,reward,risk,focuspast,focuspresent,focusfuture,relativ,motion,space,time,work,leisure,home,money,relig,death,informal,swear,netspeak,assent,nonflu,filler,AllPunc,Period,Comma,Colon,SemiC,QMark,Exclam,Dash,Quote,Apostro,Parenth,OtherP
count,1600.000000,1600,1600.000000,1600,1600,1600.000000,1600,1600,1600,1600,1600,1600,1600,1600.000000,1600.000000,1600.000000,1600.0,1600.0,1600.000000,1600.0,1600.000000,1600.000000,1600.0,1600.000000,1600.0,1600.0,1600.0,1600.0,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1600.00000,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1600.00000,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1600.00000,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1600.000000,1600.0,1600.000000,1600.000000,1600.000000
unique,NaN,20,NaN,3,1596,NaN,1596,1596,1596,1596,1596,1596,1596,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,allegro,NaN,MTurk,My daughter and I woke in the morning wanting ...,NaN,the omni was chosen for it's location whichwor...,the omni was chosen for it s location whichwor...,the omni was chosen for it s location whichwor...,my daughter and i woke in the morning wanting ...,i d been search for a cool non chain hotel for...,disappointed stay chicago monoco stay ma...,omni wa chosen locat whichwork perfectli ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,80,NaN,800,2,NaN,2,2,2,2,2,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [10]:
liwc = hotels.loc[:, "WC":"OtherP"]

In [11]:
liwc.head(2)

,WC,Analytic,Clout,Authentic,Tone,WPS,Sixltr,Dic,function,pronoun,ppron,i,we,you,shehe,they,ipron,article,prep,auxverb,adverb,conj,negate,verb,adj,compare,interrog,number,quant,affect,posemo,negemo,anx,anger,sad,social,family,friend,female,male,cogproc,insight,cause,discrep,tentat,certain,differ,percept,see,hear,feel,bio,body,health,sexual,ingest,drives,affiliation,achieve,power,reward,risk,focuspast,focuspresent,focusfuture,relativ,motion,space,time,work,leisure,home,money,relig,death,informal,swear,netspeak,assent,nonflu,filler,AllPunc,Period,Comma,Colon,SemiC,QMark,Exclam,Dash,Quote,Apostro,Parenth,OtherP
0,107,61.61,61.05,35.01,77.39,13.38,13.08,82.24,49.53,6.54,3.74,0.93,1.87,0.93,0.0,0.0,2.80,7.48,10.28,10.28,9.35,6.54,1.87,12.15,3.74,0.00,0.00,4.67,0.93,4.67,3.74,0.93,0.0,0.0,0.0,6.54,0.93,0.0,0.0,0.0,5.61,0.0,0.0,0.93,0.00,0.93,3.74,0.00,0.00,0.0,0.0,3.74,0.0,0.0,0.0,3.74,6.54,3.74,0.93,0.93,1.87,0.0,9.35,1.87,0.0,14.95,1.87,8.41,6.54,0.93,3.74,2.80,1.87,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.82,11.21,1.87,0.0,0.0,0.0,0.0,0.0,0,0.0,3.74,0.00
1,44,97.59,41.03,19.82,94.75,8.80,29.55,77.27,31.82,2.27,0.00,0.00,0.00,0.00,0.0,0.0,2.27,6.82,13.64,4.55,4.55,2.27,0.00,6.82,13.64,6.82,2.27,2.27,6.82,4.55,4.55,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,4.55,0.0,0.0,0.00,2.27,0.00,2.27,6.82,6.82,0.0,0.0,4.55,0.0,0.0,0.0,4.55,4.55,0.00,0.00,2.27,2.27,0.0,6.82,0.00,0.0,13.64,2.27,11.36,0.00,4.55,4.55,6.82,2.27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.00,11.36,9.09,0.0,0.0,0.0,0.0,0.0,0,0.0,0.00,4.55


In [12]:
liwc.columns

Index(['WC', 'Analytic', 'Clout', 'Authentic', 'Tone', 'WPS', 'Sixltr', 'Dic',
       'function', 'pronoun', 'ppron', 'i', 'we', 'you', 'shehe', 'they',
       'ipron', 'article', 'prep', 'auxverb', 'adverb', 'conj', 'negate',
       'verb', 'adj', 'compare', 'interrog', 'number', 'quant', 'affect',
       'posemo', 'negemo', 'anx', 'anger', 'sad', 'social', 'family', 'friend',
       'female', 'male', 'cogproc', 'insight', 'cause', 'discrep', 'tentat',
       'certain', 'differ', 'percept', 'see', 'hear', 'feel', 'bio', 'body',
       'health', 'sexual', 'ingest', 'drives', 'affiliation', 'achieve',
       'power', 'reward', 'risk', 'focuspast', 'focuspresent', 'focusfuture',
       'relativ', 'motion', 'space', 'time', 'work', 'leisure', 'home',
       'money', 'relig', 'death', 'informal', 'swear', 'netspeak', 'assent',
       'nonflu', 'filler', 'AllPunc', 'Period', 'Comma', 'Colon', 'SemiC',
       'QMark', 'Exclam', 'Dash', 'Quote', 'Apostro', 'Parenth', 'OtherP'],
      dtype='o

In [13]:
len(liwc.columns)

93

__1. Choose input data and target__

In [14]:
X = liwc

In [15]:
y = hotels["deceptive"]

__2. Split data__

In [16]:
X_train, X_test, y_train, y_test = split_train_test_data(X, y)

__3. Build pipeline__

In [17]:
steps_svm = [
    ('scale', StandardScaler()),
    ('clf', SVC()),
]

pipe_svm = Pipeline(steps_svm)

__4. Construct parameters for cross validation testing__

In [18]:
param_grid_svm = [
    {       
        'clf__kernel': ['rbf'], 'clf__C': [10, 100], 
         'clf__gamma': [0.01, 0.1]}
]

__5. Apply Grid Serach CV__

In [19]:
svm_grid_search = apply_grid_search_cv(pipe_svm, param_grid_svm, X_train, y_train, X_test, y_test)

C:\Users\Susan\Anaconda3\envs\thesis\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\Susan\Anaconda3\envs\thesis\lib\site-packages\sklearn\base.py:465: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
C:\Users\Susan\Anaconda3\envs\thesis\lib\site-packages\sklearn\pipeline.py:331: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


done in 3.958s
best params:
{'clf__C': 100, 'clf__gamma': 0.01, 'clf__kernel': 'rbf'}
Best cross-validation score: 0.771
Test-set score: 0.750


__6. Save Classification Report to File__

In [20]:
save_class_report_cv(svm_grid_search, X_test, y_test, ["truthful", "deceptive"], "output/svm_liwc1_report.csv")

C:\Users\Susan\Anaconda3\envs\thesis\lib\site-packages\sklearn\pipeline.py:331: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


,f1-score,precision,recall,support
truthful,0.75,0.75,0.75,240.0
deceptive,0.75,0.75,0.75,240.0
micro avg,0.75,0.75,0.75,480.0
macro avg,0.75,0.75,0.75,480.0
weighted avg,0.75,0.75,0.75,480.0


In [21]:
save_cross_validation_results(svm_grid_search, "output/svm_liwc1_validation_res.csv", True)

  clf__C clf__gamma clf__kernel mean_test_score std_test_score  \
3    100       0.01         rbf        0.770545      0.0209934   
1     10       0.01         rbf        0.769851      0.0204686   
2     10        0.1         rbf        0.626371      0.0921483   
4    100        0.1         rbf        0.626371      0.0921483   

  rank_test_score mean_fit_time  
3               1      0.176827  
1               2       0.18311  
2               3      0.186729  
4               3      0.147308  


__7. Save model to file__

In [22]:
from sklearn.externals import joblib
joblib.dump(svm_grid_search.best_estimator_, 'output/svm_liwc1.pkl')

['output/svm_liwc1.pkl']

__8. Sentiment Analysis__

In [23]:
y = hotels["polarity"]
X_train, X_test, y_train, y_test = split_train_test_data(X, y)
svm_grid_search_p = apply_grid_search_cv(pipe_svm, param_grid_svm, X_train, y_train, X_test, y_test)

done in 0.942s
best params:
{'clf__C': 10, 'clf__gamma': 0.01, 'clf__kernel': 'rbf'}
Best cross-validation score: 0.892
Test-set score: 0.873


C:\Users\Susan\Anaconda3\envs\thesis\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\Susan\Anaconda3\envs\thesis\lib\site-packages\sklearn\base.py:465: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
C:\Users\Susan\Anaconda3\envs\thesis\lib\site-packages\sklearn\pipeline.py:331: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


In [24]:
save_cross_validation_results(svm_grid_search_p, "output/svm_liwc_sent_validation_res.csv")

  clf__C clf__gamma clf__kernel mean_test_score std_test_score  \
1     10       0.01         rbf        0.892358      0.0136111   
3    100       0.01         rbf        0.892179      0.0137926   
2     10        0.1         rbf        0.768246       0.018295   
4    100        0.1         rbf        0.768246       0.018295   

  rank_test_score mean_fit_time  
1               1      0.112298  
3               2      0.103919  
2               3      0.180516  
4               3      0.160171  


In [25]:
save_class_report_cv(svm_grid_search_p, X_test, y_test, ["negative", "positive"], "output/svm_liwc_report_sent.csv")

C:\Users\Susan\Anaconda3\envs\thesis\lib\site-packages\sklearn\pipeline.py:331: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


,f1-score,precision,recall,support
negative,0.873181,0.871369,0.875000,240.0
positive,0.872651,0.874477,0.870833,240.0
micro avg,0.872917,0.872917,0.872917,480.0
macro avg,0.872916,0.872923,0.872917,480.0
weighted avg,0.872916,0.872923,0.872917,480.0


__Combined TFIDF with LIWC Analysis__

In [26]:
hotels_comb = pd.concat([hotels["norm"], liwc], axis=1)
hotels_comb.head(2)

,norm,WC,Analytic,Clout,Authentic,Tone,WPS,Sixltr,Dic,function,pronoun,ppron,i,we,you,shehe,they,ipron,article,prep,auxverb,adverb,conj,negate,verb,adj,compare,interrog,number,quant,affect,posemo,negemo,anx,anger,sad,social,family,friend,female,male,cogproc,insight,cause,discrep,tentat,certain,differ,percept,see,hear,feel,bio,body,health,sexual,ingest,drives,affiliation,achieve,power,reward,risk,focuspast,focuspresent,focusfuture,relativ,motion,space,time,work,leisure,home,money,relig,death,informal,swear,netspeak,assent,nonflu,filler,AllPunc,Period,Comma,Colon,SemiC,QMark,Exclam,Dash,Quote,Apostro,Parenth,OtherP
0,we stayed for a one night getaway with family ...,107,61.61,61.05,35.01,77.39,13.38,13.08,82.24,49.53,6.54,3.74,0.93,1.87,0.93,0.0,0.0,2.80,7.48,10.28,10.28,9.35,6.54,1.87,12.15,3.74,0.00,0.00,4.67,0.93,4.67,3.74,0.93,0.0,0.0,0.0,6.54,0.93,0.0,0.0,0.0,5.61,0.0,0.0,0.93,0.00,0.93,3.74,0.00,0.00,0.0,0.0,3.74,0.0,0.0,0.0,3.74,6.54,3.74,0.93,0.93,1.87,0.0,9.35,1.87,0.0,14.95,1.87,8.41,6.54,0.93,3.74,2.80,1.87,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.82,11.21,1.87,0.0,0.0,0.0,0.0,0.0,0,0.0,3.74,0.00
1,triple a rate with upgrade to view room was le...,44,97.59,41.03,19.82,94.75,8.80,29.55,77.27,31.82,2.27,0.00,0.00,0.00,0.00,0.0,0.0,2.27,6.82,13.64,4.55,4.55,2.27,0.00,6.82,13.64,6.82,2.27,2.27,6.82,4.55,4.55,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,4.55,0.0,0.0,0.00,2.27,0.00,2.27,6.82,6.82,0.0,0.0,4.55,0.0,0.0,0.0,4.55,4.55,0.00,0.00,2.27,2.27,0.0,6.82,0.00,0.0,13.64,2.27,11.36,0.00,4.55,4.55,6.82,2.27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.00,11.36,9.09,0.0,0.0,0.0,0.0,0.0,0,0.0,0.00,4.55


__1. Choose input data and target__

In [27]:
X = hotels_comb
y = hotels["deceptive"]

__2. Split data__

In [28]:
X_train, X_test, y_train, y_test = split_train_test_data(X, y)

__3. Build pipeline__

__4. Construct parameters for cross validation testing__

In [29]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion

In [30]:
class ItemExcluder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_frame):
        #df = data_frame.loc[:, data_frame.columns != self.key]
        df = data_frame.drop([self.key], axis=1)
        return df
    
class ItemSelector(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_frame):
        df = data_frame[self.key]
        return df

In [31]:
steps_combined = [
    ('preprocess', FeatureUnion(
        transformer_list=[
            ('terms', Pipeline([
                ('selector', ItemSelector(key='norm')),
                ('vect', CountVectorizer()),                
                ('tfidf', TfidfTransformer())
            ])),
            ('liwc', Pipeline([
                ('selector', ItemExcluder(key='norm')),         
                ('scale', StandardScaler())
            ]))                        
        ]    
    )),        
    ('clf', SVC())
]

pipe_comb = Pipeline(steps_combined)

In [32]:
param_grid_comb = [
    {'preprocess__terms__vect__stop_words': ['english', None],
        'preprocess__terms__vect__min_df': [1, 2], 
        'preprocess__terms__vect__ngram_range': [(1, 2), (1, 3)],    
        'clf__kernel': ['rbf'], 
         'clf__gamma': [0.01, 0.1],
         'clf__C': [1, 10, 100]}
]

__5. Apply Grid Serach CV__

In [33]:
svm_comb_grid_search = apply_grid_search_cv(pipe_comb, param_grid_comb, X_train, y_train, X_test, y_test)

C:\Users\Susan\Anaconda3\envs\thesis\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\Susan\Anaconda3\envs\thesis\lib\site-packages\sklearn\base.py:465: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)


done in 153.009s
best params:
{'clf__C': 1, 'clf__gamma': 0.01, 'clf__kernel': 'rbf', 'preprocess__terms__vect__min_df': 2, 'preprocess__terms__vect__ngram_range': (1, 3), 'preprocess__terms__vect__stop_words': 'english'}
Best cross-validation score: 0.789


C:\Users\Susan\Anaconda3\envs\thesis\lib\site-packages\sklearn\pipeline.py:451: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


Test-set score: 0.799


__6. Save Reports to File__

In [34]:
save_cross_validation_results(svm_comb_grid_search, "output/svm_lwic_comb1_validation_res.csv", True)

  clf__C clf__gamma clf__kernel preprocess__terms__vect__min_df  \
7      1       0.01         rbf                               2   
5      1       0.01         rbf                               2   
6      1       0.01         rbf                               2   
1      1       0.01         rbf                               1   
2      1       0.01         rbf                               1   
3      1       0.01         rbf                               1   

  preprocess__terms__vect__ngram_range preprocess__terms__vect__stop_words  \
7                               (1, 3)                             english   
5                               (1, 2)                             english   
6                               (1, 2)                                None   
1                               (1, 2)                             english   
2                               (1, 2)                                None   
3                               (1, 3)                        

In [35]:
save_class_report_cv(svm_comb_grid_search, X_test, y_test, ["truthful", "deceptive"], "output/svm_lwic_comb1_report.csv")

C:\Users\Susan\Anaconda3\envs\thesis\lib\site-packages\sklearn\pipeline.py:451: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


,f1-score,precision,recall,support
truthful,0.783550,0.815315,0.754167,240.0
deceptive,0.799197,0.771318,0.829167,240.0
micro avg,0.791667,0.791667,0.791667,480.0
macro avg,0.791373,0.793317,0.791667,480.0
weighted avg,0.791373,0.793317,0.791667,480.0


__7. Save model to file__

In [36]:
from sklearn.externals import joblib
joblib.dump(svm_comb_grid_search.best_estimator_, 'output/svm_lwic_comb1.pkl')

['output/svm_lwic_comb1.pkl']

__8. Sentiment__

In [37]:
y = hotels["polarity"]
X_train, X_test, y_train, y_test = split_train_test_data(X, y)
grid_search_p = apply_grid_search_cv(pipe_comb, param_grid_comb, X_train, y_train, X_test, y_test)


C:\Users\Susan\Anaconda3\envs\thesis\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\Susan\Anaconda3\envs\thesis\lib\site-packages\sklearn\base.py:465: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)


done in 125.580s
best params:
{'clf__C': 1, 'clf__gamma': 0.01, 'clf__kernel': 'rbf', 'preprocess__terms__vect__min_df': 1, 'preprocess__terms__vect__ngram_range': (1, 2), 'preprocess__terms__vect__stop_words': 'english'}
Best cross-validation score: 0.904
Test-set score: 0.892


C:\Users\Susan\Anaconda3\envs\thesis\lib\site-packages\sklearn\pipeline.py:451: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


In [38]:
save_cross_validation_results(grid_search_p, "output/svm_lwic_comb1_validation_res_sent.csv", True)

  clf__C clf__gamma clf__kernel preprocess__terms__vect__min_df  \
1      1       0.01         rbf                               1   
2      1       0.01         rbf                               1   
3      1       0.01         rbf                               1   
4      1       0.01         rbf                               1   
5      1       0.01         rbf                               2   
6      1       0.01         rbf                               2   

  preprocess__terms__vect__ngram_range preprocess__terms__vect__stop_words  \
1                               (1, 2)                             english   
2                               (1, 2)                                None   
3                               (1, 3)                             english   
4                               (1, 3)                                None   
5                               (1, 2)                             english   
6                               (1, 2)                        

In [39]:
save_class_report_cv(grid_search_p, X_test, y_test, ["negative", "positive"], "output/svm_lwic_comb1_report_sent.csv")

C:\Users\Susan\Anaconda3\envs\thesis\lib\site-packages\sklearn\pipeline.py:451: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


,f1-score,precision,recall,support
negative,0.895277,0.882591,0.908333,240.0
positive,0.892178,0.905579,0.879167,240.0
micro avg,0.893750,0.893750,0.893750,480.0
macro avg,0.893727,0.894085,0.893750,480.0
weighted avg,0.893727,0.894085,0.893750,480.0
